In [1]:
import numpy as np
import pandas as pd
import re

#### 下载词库，用于词形还原，参考https://www.cnblogs.com/pinard/p/6756534.html

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
#nltk.download()

#### 去掉训练集的标点符号

In [2]:
labels = []
with open(r'C:\Users\Administrator\Desktop\all\AI\project\project_data\2\trainLabel.txt', 'r', encoding='utf-8') as trainLabel:
    for label in trainLabel:
        labels.append(int(re.sub(r'\n', '', label)))
labels = np.array(labels)
#train = pd.DataFrame({'paragraph': sentences, 'label': labels})

#### 转化为小写

In [219]:
for sentence in sentences:
    for i, word in enumerate(sentence):
        sentence[i] = word.lower()

#### 词形还原的效果不好，无法识别不同时态，采用词干提取效果较好

In [131]:
stemmer = SnowballStemmer("english") # Choose a language
stemmer.stem("play") # Stem a word
stemmer.stem("plays") # Stem a word
stemmer.stem("played") # Stem a word

'play'

'play'

'play'

In [221]:
for sentence in sentences:
    for i, word in enumerate(sentence):
        sentence[i] = stemmer.stem(word)

#### 读入停用词，去掉停用词 (去停用词后，词干提取，再次去停用词)

In [145]:
stop_word_list2 = []
with open('/home/inpluszyc/course/AI/project_data/2/stop_word2.txt', 'r', encoding='utf-8') as st:
    stop_word_list2 = st.read().split('\n')

In [222]:
for sentence in sentences:
    for word in sentence.copy():
        if word in stop_word_list2:
            sentence.remove(word)
len(sentences[1])

149

#### 保存处理后的结果

In [223]:
import pickle
with open('/home/inpluszyc/course/AI/project_data/2/new_train.pkl', 'wb') as nt:
    pickle.dump(sentences, nt)

#### 获取处理后的训练文本

In [3]:
import pickle
with open(r'C:\Users\Administrator\Desktop\all\AI\project\project_data\2\new_train.pkl', 'rb') as nt:
    sentences = pickle.load(nt)

#### 获取所有单词

In [4]:
all_word = []
for sentence in sentences:
    for word in sentence:
        if word not in all_word:
            all_word.append(word)
all_word

['3d',
 'technolog',
 '1950',
 'anim',
 'rememb',
 'donald',
 'duck',
 'cartoon',
 'chip',
 'dale',
 'moment',
 'plot',
 'circus',
 'feed',
 'eleph',
 'peanut',
 'steal',
 'watch',
 '1960',
 'happen',
 'meet',
 'robinson',
 'theater',
 'movi',
 'explain',
 'detail',
 'origin',
 '100',
 'view',
 'glass',
 'claim',
 'bad',
 'move',
 'difficult',
 'prove',
 'wrong',
 'top',
 'william',
 'shakespear',
 'merchant',
 'venic',
 'portray',
 '16th',
 'centuri',
 'al',
 'pacino',
 'play',
 'shylock',
 'jewish',
 'loan',
 'shark',
 'reveng',
 'cathol',
 'slow',
 'build',
 'hour',
 'film',
 'charact',
 'hear',
 'jew',
 'previous',
 'mobster',
 'type',
 'role',
 'credit',
 'costum',
 'design',
 'director',
 'fit',
 'time',
 'period',
 'renaiss',
 'envis',
 'anti-semit',
 'vibe',
 'minut',
 'advantag',
 'lock',
 'ghetto',
 'regular',
 'job',
 'comparison',
 'passion',
 'christ',
 'peopl',
 'believ',
 'holiday',
 'mistreat',
 'receiv',
 'locat',
 'shot',
 'sceneri',
 'beauti',
 'ugli',
 'depend',
 's

In [43]:
len(all_word)

92995

In [8]:
len(all_word)

92995

#### 使用卡方检验或信息增益进行特征选择https://www.cnblogs.com/june0507/p/7601001.html

In [7]:
train = pd.DataFrame({'sentences': sentences, 'label': labels})

##### 信息增益

In [5]:
#计算TF矩阵
tf = np.zeros((len(sentences), len(all_word)), dtype=bool) #(24000,92995)
for i, sentence in enumerate(sentences):
    for word in sentence:
        index = all_word.index(word)
        tf[i][index] = 1

In [6]:
def calc_info_gain(i, labels):
    tf_i = tf[:, i]
    pos, neg = tf_i[labels==1], tf_i[labels==0]
    A = pos.sum() #包含特征且为正样本
    B = neg.sum() #包含特征且为负样本
    C = len(pos) - A #不包含特征且为正样本
    D = len(neg) - B #不包含特征且为负样本
    print(A,B,C,D)
    if A+B <= 5: #减少低频性缺陷
        return 0 
    res = (A + B) * calc_entropy([A,B]) + (C + D) * calc_entropy([C,D])
    return res

In [7]:
import queue
Q = queue.PriorityQueue()
for i in range(len(all_word)):
    Q.put((calc_info_gain(i, labels), i)) #小项堆排序

16 18 12005 11961


NameError: name 'calc_entropy' is not defined

In [ ]:
all_word_by_gain = []
for i in range(10000):
    all_word_by_gain.append(all_word[Q.get()[1]])
len(all_word_by_gain)
all_word_by_gain.append('None')

##### 对给定词进行卡方检验

In [44]:
def calc_chi_square(i, labels):
    tf_i = tf[:, i]
    pos, neg = tf_i[labels==1], tf_i[labels==0]
    A = pos.sum() #包含特征且为正样本
    B = neg.sum() #包含特征且为负样本
    C = len(pos) - A #不包含特征且为正样本
    D = len(neg) - B #不包含特征且为负样本
    print(A,B,C,D)
    if A+B <= 50: #减少低频性缺陷
        return 0 
    kf = (A * D - B * C) ** 2 / ((A+B)*(C+D))
    return kf

In [45]:
import queue
Q = queue.PriorityQueue()
for i in range(len(all_word)):
    Q.put((-calc_chi_square(i, labels), i)) #取负号，小项堆排序

16 18 12005 11961
108 97 11913 11882
85 50 11936 11929
607 524 11414 11455
915 657 11106 11322
96 65 11925 11914
41 40 11980 11939
201 213 11820 11766
33 38 11988 11941
18 8 12003 11971
1173 953 10848 11026
1775 2942 10246 9037
36 29 11985 11950
49 74 11972 11905
40 33 11981 11946
12 11 12009 11968
304 237 11717 11742
4085 4645 7936 7334
48 43 11973 11936
913 1089 11108 10890
773 541 11248 11438
47 13 11974 11966
371 423 11650 11556
7121 8317 4900 3662
330 516 11691 11463
465 298 11556 11681
1179 1367 10842 10612
97 153 11924 11826
1055 663 10966 11316
114 97 11907 11882
206 228 11815 11751
1443 4427 10578 7552
1177 803 10844 11176
365 246 11656 11733
425 352 11596 11627
600 928 11421 11051
786 641 11235 11338
398 242 11623 11737
103 68 11918 11911
22 17 11999 11962
36 9 11985 11970
969 554 11052 11425
9 6 12012 11973
295 176 11726 11803
110 89 11911 11890
52 38 11969 11941
3079 2355 8942 9624
8 3 12013 11976
77 59 11944 11920
28 11 11993 11968
33 59 11988 11920
167 200 11854 11779
39 

23 68 11998 11911
59 337 11962 11642
11 7 12010 11972
1 6 12020 11973
0 7 12021 11972
18 38 12003 11941
138 131 11883 11848
21 41 12000 11938
172 289 11849 11690
0 1 12021 11978
343 237 11678 11742
0 1 12021 11978
962 424 11059 11555
614 601 11407 11378
224 169 11797 11810
32 142 11989 11837
70 68 11951 11911
296 581 11725 11398
193 131 11828 11848
35 95 11986 11884
14 10 12007 11969
0 2 12021 11977
0 1 12021 11978
0 1 12021 11978
74 56 11947 11923
5 3 12016 11976
985 1485 11036 10494
26 141 11995 11838
6 5 12015 11974
4 1 12017 11978
57 45 11964 11934
465 283 11556 11696
7 9 12014 11970
0 1 12021 11978
462 353 11559 11626
271 358 11750 11621
172 161 11849 11818
41 43 11980 11936
17 21 12004 11958
100 37 11921 11942
22 6 11999 11973
0 2 12021 11977
126 125 11895 11854
12 8 12009 11971
1 5 12020 11974
268 253 11753 11726
120 100 11901 11879
105 54 11916 11925
172 345 11849 11634
43 142 11978 11837
125 691 11896 11288
49 40 11972 11939
129 307 11892 11672
0 1 12021 11978
507 380 11514 11

12 15 12009 11964
1 0 12020 11979
244 207 11777 11772
9 18 12012 11961
171 272 11850 11707
26 7 11995 11972
6 6 12015 11973
581 334 11440 11645
13 6 12008 11973
872 512 11149 11467
3 0 12018 11979
11 2 12010 11977
216 119 11805 11860
272 179 11749 11800
308 213 11713 11766
51 40 11970 11939
185 178 11836 11801
127 96 11894 11883
3 0 12018 11979
395 320 11626 11659
44 40 11977 11939
428 246 11593 11733
262 272 11759 11707
6 1 12015 11978
158 102 11863 11877
275 218 11746 11761
6 0 12015 11979
94 78 11927 11901
1 1 12020 11978
145 72 11876 11907
137 84 11884 11895
24 8 11997 11971
481 400 11540 11579
366 219 11655 11760
167 136 11854 11843
737 867 11284 11112
247 226 11774 11753
62 18 11959 11961
44 39 11977 11940
24 19 11997 11960
875 256 11146 11723
2 2 12019 11977
40 28 11981 11951
604 372 11417 11607
87 92 11934 11887
85 96 11936 11883
219 161 11802 11818
192 159 11829 11820
254 148 11767 11831
60 32 11961 11947
255 232 11766 11747
39 51 11982 11928
127 66 11894 11913
169 267 11852 1

164 167 11857 11812
148 123 11873 11856
401 416 11620 11563
798 707 11223 11272
161 84 11860 11895
427 263 11594 11716
4 1 12017 11978
254 337 11767 11642
30 24 11991 11955
10 6 12011 11973
39 20 11982 11959
45 33 11976 11946
48 47 11973 11932
51 63 11970 11916
35 49 11986 11930
81 387 11940 11592
0 1 12021 11978
0 1 12021 11978
217 165 11804 11814
0 10 12021 11969
28 71 11993 11908
329 283 11692 11696
888 436 11133 11543
80 68 11941 11911
260 190 11761 11789
53 168 11968 11811
24 37 11997 11942
136 379 11885 11600
50 175 11971 11804
157 203 11864 11776
54 37 11967 11942
465 732 11556 11247
412 241 11609 11738
16 28 12005 11951
19 14 12002 11965
103 119 11918 11860
130 105 11891 11874
421 249 11600 11730
300 277 11721 11702
89 114 11932 11865
596 468 11425 11511
7 7 12014 11972
111 197 11910 11782
63 72 11958 11907
118 67 11903 11912
523 631 11498 11348
6 1 12015 11978
820 600 11201 11379
65 84 11956 11895
18 25 12003 11954
101 99 11920 11880
115 121 11906 11858
4 18 12017 11961
17 15 

46 35 11975 11944
7 3 12014 11976
48 34 11973 11945
72 114 11949 11865
18 15 12003 11964
2 0 12019 11979
1 2 12020 11977
27 7 11994 11972
51 43 11970 11936
289 158 11732 11821
9 3 12012 11976
1 0 12020 11979
27 29 11994 11950
1 0 12020 11979
221 74 11800 11905
59 32 11962 11947
59 31 11962 11948
165 277 11856 11702
9 6 12012 11973
170 107 11851 11872
67 59 11954 11920
26 20 11995 11959
2 4 12019 11975
78 112 11943 11867
308 69 11713 11910
282 328 11739 11651
12 6 12009 11973
560 412 11461 11567
1 0 12020 11979
87 16 11934 11963
3 0 12018 11979
2 1 12019 11978
1 0 12020 11979
3 0 12018 11979
26 12 11995 11967
12 17 12009 11962
14 11 12007 11968
11 11 12010 11968
30 24 11991 11955
4 0 12017 11979
426 255 11595 11724
17 4 12004 11975
10 0 12011 11979
18 8 12003 11971
37 16 11984 11963
4 7 12017 11972
14 25 12007 11954
19 22 12002 11957
95 336 11926 11643
30 20 11991 11959
44 53 11977 11926
3 0 12018 11979
41 40 11980 11939
8 2 12013 11977
250 237 11771 11742
78 58 11943 11921
117 69 11904

...

3 0 12018 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 1202

0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 1202

1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 1202

0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 1202

1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
2 0 12019 11979
2 0 12019 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
0 1 12021 11978
0 1 12021 11978
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 12020 11979
1 0 1202

In [46]:
all_word_by_kf = []
for i in range(10000):
    top = Q.get()
    if top[0] == 0: break
    all_word_by_kf.append(all_word[top[1]])

In [47]:
len(all_word_by_kf)
all_word_by_kf.append('None')

2209

#### 保存次数大于100的词

In [48]:
import pickle
with open(r'C:\Users\Administrator\Desktop\all\AI\project\project_data\2\word_kf2.pkl', 'wb') as word_kf:
    pickle.dump(all_word_by_kf, word_kf)

In [37]:
import pickle
with open(r'C:\Users\Administrator\Desktop\all\AI\project\project_data\2\word_kf2.pkl', 'rb') as word_kf:
    all_word_by_kf2 = pickle.load(word_kf)

In [38]:
len(all_word_by_kf2)

1338

#### KNN

In [13]:
sentences_train, labels_train = sentences[: int(len(sentences) * 0.9)], labels[: int(len(sentences) * 0.9)]
sentences_va, labels_va = sentences[int(len(sentences) * 0.9):], labels[int(len(sentences) * 0.9):]

In [14]:
#编码所有训练样本
train_one_hot = np.zeros((len(sentences), len(all_word_by_kf)), dtype='int8')
for i, sentence in enumerate(sentences):
    for word in sentence:
        try:
            train_one_hot[i][all_word_by_kf.index(word)] += 1
        except:
            train_one_hot[-1] += 1
#for each in train_one_hot:
#    each /= each.sum()

In [15]:
train_one_hot_09 = train_one_hot[:int(len(sentences) * 0.9)]
va_one_hot = train_one_hot[:int(len(sentences) * 0.9)]

In [16]:
def one_hot_encode(sentence, all_word_by_kf):
    one_hot = np.zeros(len(all_word_by_kf))
    for word in sentence:
        try:
            one_hot[all_word_by_kf.index(word)] += 1
        except:
            one_hot[-1] += 1
    return one_hot

def calc_distance(one_hot1, one_hot2, p):
    if p == 1: return np.sum(np.abs(one_hot1 - one_hot2))
    if p == 2: return np.sqrt(np.sum((one_hot1 - one_hot2) ** 2))
    if p == 3: return -one_hot1.dot(one_hot2) / (np.sqrt((one_hot1 ** 2).sum() * (one_hot2 ** 2).sum()))

def predict_one(train_one_hot, labels, one_hot, k, p):
    '''
    Input:
        train_one_hot是对训练集的one_hot编码
        label是对应的标签
        sentence是待预测的句子 -> one_hot
        k是邻居个数
        p是相似度计算的方法
    Output:
        预测的标签
    '''
    Q = queue.PriorityQueue()
    emotion_vote = {}
    for i, each in enumerate(train_one_hot):
        distance = calc_distance(one_hot, each, p)
        Q.put((distance, i))
    while k:
        emotion = labels[Q.get()[1]]
        emotion_vote[emotion] = emotion_vote.get(emotion, 0) + 1
        k -= 1
    for key, value in emotion_vote.items():
        if value == max(emotion_vote.values()):
            return key    

def predict_all(train_one_hot, labels, to_predict, k, p):
    y_pre = []
    for one_hot in to_predict:
        pre = predict_one(train_one_hot, labels, one_hot, k, p)
        y_pre.append(pre)
    return np.array(y_pre)

def ACC(y_pre):
    return (labels_va == y_pre).sum() / len(y_pre)

In [17]:
predict_one(train_one_hot_09, labels_train, va_one_hot[0], 5,1)

0

In [18]:
for p in range(1,4):
    for k in range(5,int(np.sqrt(len(sentences_va))) + 5):
        y_pre = predict_all(train_one_hot, labels, sentences_va, k, p)
        score = ACC(y_pre)
        print(p, k, score)
        if (score > 0.60):
            print('*****************************************')

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U14') dtype('<U14') dtype('<U14')

In [ ]:
predict_all(train_one_hot, labels_train, sentences_va[:3],5,1)

In [ ]:
labels_va[:3]

In [ ]:
#编码所有训练样本
train_one_hot2 = np.zeros((len(sentences_train), len(all_word_by_kf)), dtype='float16')
for i, sentence in enumerate(sentences_train):
    for word in sentence:
        try:
            train_one_hot2[i][all_word_by_kf.index(word)] += 1
        except:
            train_one_hot2[-1] += 1
for each in train_one_hot2:
    each /= each.sum()

In [ ]:
for p in range(1,4):
    for k in range(5,int(np.sqrt(len(sentences_va))) + 5):
        y_pre = predict_all(train_one_hot2, labels, sentences_va, k, p)
        score = ACC(y_pre)
        print(p, k, score)
        if (score > 0.60):
            print('*****************************************')

#### 决策树

In [49]:
#编码所有训练样本
train_one_hot = np.zeros((len(sentences), len(all_word_by_kf)), dtype='float16')
for i, sentence in enumerate(sentences):
    for word in sentence:
        try:
            train_one_hot[i][all_word_by_kf.index(word)] += 1
        except:
            train_one_hot[i][-1] += 1

In [50]:
train = pd.DataFrame(train_one_hot, columns=range(len(all_word_by_kf)))

In [51]:
target = pd.DataFrame({'target': labels})

In [52]:
train = pd.concat([train, target], axis=1)

In [53]:
train

,0,1,2,3,4,5,6,7,8,9,...,2201,2202,2203,2204,2205,2206,2207,2208,2209,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,3.0,8.0,75.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,25.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,42.0,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,86.0,0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,39.0,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,39.0,0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,108.0,0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,43.0,1


In [ ]:
#定义树节点
class Node():
    def __init__(self, feature, feature_divide, target, child = []):
        self.feature = feature #划分的属性
        self.feature_divide = feature_divide #属性的取值
        self.target = target #此节点判断的结果
        self.child = child

In [ ]:
#计算特征的信息增益、信息增益比、Gini指数
from itertools import *
#计算信息增益
def calc_entropy(num):
    entropy = 0
    for each in num:
        entropy += each / sum(num) * np.log2(sum(num) / each)
    return entropy

#计算gini系数
def calc_gini(num):
    num = np.array(num)
    return 1 - np.sum((num / np.sum(num)) ** 2)

#计算条件熵
def calc_condition_entropy(train, feature):
    '''
    train: 包含标签的训练数据
    feature: 待计算的特征
    '''
    values = train.groupby(feature)['target'].value_counts() #分组
    index = values.index #得到索引
    values = values.values
    res = 0
    num = [values[0]]
    for i in range(1, len(index)):
        if index[i][0] == index[i - 1][0]:
            num.append(values[i])
        else:
            res += calc_entropy(num) * sum(num) / train.shape[0]
            num = [values[i]]
    res += calc_entropy(num) * sum(num) / train.shape[0]
    return res

#划分特征的取值
def split_feature(values):
    comb = []
    for i in range(1, len(values)):
        comb.extend(list(combinations(values, i)))
    return list(zip(comb[0: len(comb) // 2], comb[len(comb) - 1: len(comb) // 2 - 1: -1]))

#遍历所有可能的二分组合，计算基尼系数            
def calc_gini_with_feature(train, feature):
    split_group = split_feature(train[feature].unique())
    min_gini = 1
    min_group = None
    for group in split_group:
        gini = 0
        in_group1 = train[feature].apply(lambda x: x in group[0])
        group1, group2 = train[in_group1], train[-in_group1]  #获得两个组划分后的数据集
        #比较得到最优的划分及基尼指数
        gini = (len(group1) * calc_gini(group1['target'].value_counts().values) 
        + len(group2) * calc_gini(group2['target'].value_counts().values)) / len(train)
        if gini < min_gini:
            min_gini, min_group = gini, group
    return [feature, -min_gini, min_group]
    
    
#得到划分的特征
def calc_criteria(train, features, tree_type='ID3'):
    '''
    train: 包含标签的训练数据
    feature: 待计算、比较的所有特征
    tree_type: 决定划分标准
    return: 返回0表示无法继续划分，否则返回划分的特征
    '''
    if len(features) == 0: return 0
    if tree_type == 'ID3' or tree_type == 'C4.5': 
        target_entropy = calc_entropy(train['target'].value_counts().values) #D的熵
    criteria = []
    #得到每个特征的标签与分类指标的值构成的列表集合
    for each in features:
        if tree_type == 'ID3':
            criteria.append([each, target_entropy - calc_condition_entropy(train, each)])
        elif tree_type == 'C4.5':
            criteria.append([each, (target_entropy - calc_condition_entropy(train, each)) / 
                             calc_entropy(train[each].value_counts().values)]) 
        elif tree_type == 'CART':
            criteria.append(calc_gini_with_feature(train, each))
    maxIndex, equal_num = 0, 0
    for i in range(len(criteria)):
        if criteria[i][1] == criteria[maxIndex][1]:
            equal_num += 1
        elif criteria[i][1] > criteria[maxIndex][1]:
            maxIndex = i
    if equal_num == len(criteria) and equal_num > 1: #若特征数大于1且在分类指标的值都相同，则无法划分
        return 0
    else:
        return criteria[maxIndex][0] if tree_type != 'CART' else (criteria[maxIndex][0], criteria[maxIndex][2]) 

In [27]:
#计算准确率
def calc_score(y_pre, y_true):
    return (y_pre == y_true).sum() / len(y_pre)

In [ ]:
class DecisionTree():
    def __init__(self, tree_type='ID3'):
        self.root = None
        self.tree_type = tree_type
    
    #递归生成树
    def get_root(self, train, features, tree_type):
        if len(train['target'].unique()) == 1: #若标签取值全部相同，则为叶子节点
            return Node(None, [], train['target'].iloc[0], None)
        mode = train['target'].mode()[0]   #计算众数
        featureChoose = calc_criteria(train, features, tree_type)
        #若特征集为空或D在所有特征上取值相同，featureChoose为0，返回叶子节点
        if featureChoose == 0: return Node(None, [], mode, None) 
        
        if tree_type != 'CART':
            index = train[featureChoose].unique()
            root = Node(featureChoose, index, mode, [None] * len(index)) #根据特征取值的类别生成相应节点
            #递归生成子节点
            features.remove(featureChoose)
            for i in range(len(index)):
                root.child[i] = DecisionTree.get_root(self,train[train[featureChoose] == index[i]], features, tree_type)
            features.append(featureChoose)
            return root
        else:
            index = featureChoose[1]
            root = Node(featureChoose[0], index, mode, [None, None]) #二分
            in_group1 = train[featureChoose[0]].apply(lambda x: x in index[0])
            for i in range(2):
                if len(index[i]) > 1:
                    root.child[i] = DecisionTree.get_root(self, train[in_group1 if i == 0 else -in_group1], features, tree_type)
                else:
                    features.remove(featureChoose[0])
                    root.child[i] = DecisionTree.get_root(self, train[in_group1 if i == 0 else -in_group1], features, tree_type)
                    features.append(featureChoose[0])
            return root
        
    def fit(self, x_train, y_train):
        features = list(x_train.columns)
        train = pd.concat([x_train, y_train], axis=1)
        self.root = DecisionTree.get_root(self, train, features, self.tree_type)
        return self

    def predict_one(self, root, one_test, tree_type):
        i = 0
        while i < len(root.feature_divide):#遍历每个属性，查看是否符合
            if tree_type != 'CART' and root.feature_divide[i] == one_test[root.feature]:
                return DecisionTree.predict_one(self, root.child[i], one_test, tree_type)
            elif tree_type == 'CART' and one_test[root.feature] in root.feature_divide[i]:
                return DecisionTree.predict_one(self, root.child[i], one_test, tree_type)
            i += 1
        if i == len(root.feature_divide):#若没有符合或属性为空的
            return root.target
    
    def predict(self, x_test):
        if (self.root == None):
            print('fit first')
            return None
        else:
            y_pre = np.zeros(len(x_test))
            for i in range(len(x_test)):
                y_pre[i] = DecisionTree.predict_one(self, self.root, x_test.iloc[i], self.tree_type)
            return y_pre
    
    def calc_leaf(self, root):
        if len(root.feature_divide) == 0:
            return 1
        num = 0
        for each in root.child:
            num += DecisionTree.calc_leaf(self, each)
        return num
    
    def post_cut_au(self, root, x_vali, y_vali):
        if len(root.feature_divide) == 0:
            return
        for each in root.child:
            DecisionTree.post_cut_au(self, each, x_vali, y_vali)
        score = calc_score(DecisionTree.predict(self, x_vali), y_vali) #原来的分数

        temp_div = root.feature_divide
        root.feature_divide = []
        score_new = calc_score(DecisionTree.predict(self, x_vali), y_vali) #新的分数
        if score_new < score:
            root.feature_divide = temp_div
    
    def post_cut(self, x_vali, y_vali):
        DecisionTree.post_cut_au(self, self.root, x_vali, y_vali)

In [ ]:
DT = DecisionTree('ID3')
DT2 = DecisionTree('C4.5')
DT3 = DecisionTree('CART')

In [54]:
train1, valid = train.iloc[:22000], train.iloc[22000:]

In [ ]:
DT.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = DT.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

In [ ]:
train.groupby(0)['target'].value_counts()

In [ ]:
valid.shape

In [ ]:
DT2.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = DT2.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

In [ ]:
DT3.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = DT3.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

#### 1339

In [55]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

0.68899999999999995

In [29]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

0.72299999999999998

In [28]:
calc_score(y_pre, valid.iloc[:,-1])

0.70199999999999996

In [30]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

0.8145

In [31]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

KeyboardInterrupt: 

In [41]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

0.80549999999999999

In [42]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

0.57650000000000001

#### 2200+

In [56]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

0.755

In [57]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

0.81850000000000001

In [58]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train1.iloc[:, :-1], train1.iloc[:, -1])
y_pre = clf.predict(valid.iloc[:, :-1])
calc_score(y_pre, valid.iloc[:,-1])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

0.80649999999999999

In [61]:
import numpy as np

class MultinomialNB2(object):
    def __init__(self,alpha=1.0,fit_prior=True,class_prior=None):
        self.alpha = alpha
        self.fit_prior = fit_prior
        self.class_prior = class_prior
        self.classes = None
        self.conditional_prob = None


    def _calculate_feature_prob(self,feature):
        values = np.unique(feature)
        total_num = float(len(feature))
        value_prob = {}
        for v in values:
            value_prob[v] = (( np.sum(np.equal(feature,v)) + self.alpha ) /( total_num + len(values)*self.alpha))
        return value_prob


    def fit(self,X,y):
        #TODO: check X,y

        self.classes = np.unique(y)
        #calculate class prior probabilities: P(y=ck)
        if self.class_prior == None:
            class_num = len(self.classes)
            if not self.fit_prior:
                self.class_prior = [1.0/class_num for _ in range(class_num)]  #uniform prior
            else:
                self.class_prior = []
                sample_num = float(len(y))
                for c in self.classes:
                    c_num = np.sum(np.equal(y,c))
                    self.class_prior.append((c_num+self.alpha)/(sample_num+class_num*self.alpha))

        #calculate Conditional Probability: P( xj | y=ck )
        self.conditional_prob = {}  # like { c0:{ x0:{ value0:0.2, value1:0.8 }, x1:{} }, c1:{...} }
        for c in self.classes:
            self.conditional_prob[c] = {}
            for i in range(len(X[0])):  #for each feature
                feature = X[np.equal(y,c)][:,i]
                self.conditional_prob[c][i] = self._calculate_feature_prob(feature)
        return self


    #given values_prob {value0:0.2,value1:0.1,value3:0.3,.. } and target_value
    #return the probability of target_value
    def _get_xj_prob(self,values_prob,target_value):
        return values_prob[target_value]

    #predict a single sample based on (class_prior,conditional_prob)
    def _predict_single_sample(self,x):
        label = -1
        max_posterior_prob = 0

        #for each category, calculate its posterior probability: class_prior * conditional_prob
        for c_index in range(len(self.classes)):
            current_class_prior = self.class_prior[c_index]
            current_conditional_prob = 1.0
            feature_prob = self.conditional_prob[self.classes[c_index]]
            j = 0
            for feature_i in feature_prob.keys():
                current_conditional_prob *= self._get_xj_prob(feature_prob[feature_i],x[j])
                j += 1

            #compare posterior probability and update max_posterior_prob, label
            if current_class_prior * current_conditional_prob > max_posterior_prob:
                max_posterior_prob = current_class_prior * current_conditional_prob
                label = self.classes[c_index]
        return label

    #predict samples (also single sample)           
    def predict(self,X):
        #TODO1:check and raise NoFitError 
        #ToDO2:check X
        if X.ndim == 1:
                return self._predict_single_sample(X)
        else:
                #classify each sample   
                labels = []
                for i in range(X.shape[0]):
                        label = self._predict_single_sample(X[i])
                        labels.append(label)
                return labels

In [63]:
clf = MultinomialNB2()
clf.fit(train1.iloc[:, :-1].values, train1.iloc[:, -1].values)
y_pre = clf.predict(valid.iloc[:, :-1].values)
calc_score(y_pre, valid.iloc[:,-1])

KeyError: 15.0

In [69]:
def pre(clf, test):
    y_pre = []
    for i in range(len(test)):
        print(i, clf._predict_single_sample(test[i]))
        y_pre.append(clf._predict_single_sample(test[i]))
    return y_pre
                     
y_pre = pre(clf, valid.iloc[:, :-1].values)

0 0
1 0
2 0


KeyError: 15.0

In [71]:
clf._predict_single_sample(valid.iloc[:, :-1].values[3])

KeyError: 15.0